In [25]:
from pprint import pprint
from decaylanguage import *
from phasespace import GenParticle
from particle import Particle

DecayChainViewer only contains one way a particle can decay.
The best way to handle everything (at least for now) might therefore be via dicts.
One can also directly convert DecayChains to GenParticle, since they have a one-to-one format.
This is because DecayChain only contains one way the mother can decay, which phasespace is built on.
Not sure if this is intended.
TODO: Ask in Gitter?

Example:

In [3]:
parser = DecFileParser('test_example_Dst.dec')
parser.parse()

In [4]:
d = parser.build_decay_chains('D+')
dc = DecayChain.from_dict(d)
pprint(dc.to_dict())
pprint(d)

{'D+': [{'bf': 1.0,
         'fs': ['K-',
                'pi+',
                'pi+',
                {'pi0': [{'bf': 6.5e-08,
                          'fs': ['e+', 'e-'],
                          'model': 'PHSP',
                          'model_params': ''}]}],
         'model': 'PHSP',
         'model_params': ''}]}
{'D+': [{'bf': 1.0,
         'fs': ['K-',
                'pi+',
                'pi+',
                {'pi0': [{'bf': 0.988228297,
                          'fs': ['gamma', 'gamma'],
                          'model': 'PHSP',
                          'model_params': ''},
                         {'bf': 0.011738247,
                          'fs': ['e+', 'e-', 'gamma'],
                          'model': 'PI0_DALITZ',
                          'model_params': ''},
                         {'bf': 3.3392e-05,
                          'fs': ['e+', 'e+', 'e-', 'e-'],
                          'model': 'PHSP',
                          'model_params': ''},
            

Clearly, there is a difference between the 2 above.

The format of the output dict is (according to the `parser.build_decay_chains` docstring):
```python
{mother: [{'bf': float, 'fs': list, 'model': str, 'model_params': str}]}
```
where
- 'bf' stands for the decay mode branching fraction
- 'fs' is a list of final-state particle names (strings) and/or dictionaries of the same form as the decay chain above
- 'model' is the model name, if found, else ''
- 'model_params' are the model parameters, if specified, else ''

In [6]:
dm1 = DecayMode(0.000043300 , 'K*0 gamma', model='HELAMP', model_params=[1.0, 0.0, 1.0, 0.0])
dm2 = DecayMode(0.6657, 'K+ pi-', model='VSS')
dc = DecayChain('B0', {'B0':dm1, 'K*0':dm2})
dc

<DecayChain: B0 -> K*0 gamma (1 sub-decays), BF=4.33e-05>

In [14]:
import tensorflow as tf
import numpy as np


def recursively_traverse(decaychain: dict) -> GenParticle:
    """Create a random GenParticle by recursively traversing a dict from decaylanguage.

    Parameters
    ----------
    decaychain: dict

    Returns
    -------
    particle : GenParticle
        The generated particle

    Notes
    -----
    This implementation is slow since it
    - gets the particle mass every time from the particle package
    - relies on an external python for-loop
    TODO: cache results of input GenParticle to make it faster and work at all.
    Decorator? Class?
    """
    mother = list(decaychain.keys())[0] # Get the only key inside the dict
    decaymodes = decaychain[mother]
    # TODO: replace with tnp in the future
    # TODO: make multiple choices at once in a future version
    i = np.random.choice(range(len(decaymodes)), p=[dm['bf'] for dm in decaymodes])
    daughter_particles = decaymodes[i]['fs']
    daughter_gens = []
    for daughter_name in daughter_particles:
        # TODO: implement mass distribution function here
        if isinstance(daughter_name, str):
            daughter = GenParticle(daughter_name, Particle.from_string(daughter_name).mass)
        elif isinstance(daughter_name, dict):
            daughter = recursively_traverse(daughter_name)
        else:
            raise TypeError(f'Expected elements in decaychain["fs"] to only be str or dict '
                            f'but found of type {type(daughter_name)}')
        daughter_gens.append(daughter)

    return GenParticle(mother, Particle.from_string(mother).mass).set_children(*daughter_gens)


def generate_nbody_naive(decaychain: dict, nevents: int) -> list[dict[str, tf.Tensor]]:
    """Generate events from a full decay
    Parameters
    ----------
    decaychain : dict
        Dict describing a decay.
        Can contain multiple different ways that a particle can decay in.
    nevents : int
        Number of events that should be generated

    Returns
    -------
    events : list[dict[str, tf.Tensor]]
        list of all the generated events and the four-momenta of the final state particles.
    Notes
    -----
    This implementation is very slow and inefficient.
    A better version will be made later.
    """
    events = []
    for i in range(nevents):
        particle = recursively_traverse(decaychain)
        events.append(particle.generate(1))

    return events

recursively_traverse(d)

KeyError: "Particle name {'gamma'} already used"

In [16]:
gamma = GenParticle('gamma', 0)
GenParticle('D+', 500).set_children(
    GenParticle('K-', 100),
    GenParticle('pi+', 50),
    GenParticle('pi-', 50),
    GenParticle('pi0', 50).set_children(
        gamma,
        gamma
    )
)


KeyError: "Particle name {'gamma'} already used"

In [26]:
p = GenParticle('D+', 500).set_children(GenParticle('K-', 100), GenParticle('K+', 100))
print(*p.children)
p.children = [GenParticle('e+', 100), GenParticle('e+', 100)]
print(*p.children)
p.generate(10)


<phasespace.GenParticle: name='K-' mass=100.00 children=[]> <phasespace.GenParticle: name='K+' mass=100.00 children=[]>
<phasespace.GenParticle: name='e+' mass=100.00 children=[]> <phasespace.GenParticle: name='e+' mass=100.00 children=[]>


NotImplementedError: Cannot convert a symbolic Tensor (n_events:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported